# The netCDF file format

* popular scientific file format for ocean and atmospere gridded datasets
* netCDF is a collection of formats for storing arrays

    * netCDF classic
        * more widespread
        * 2 GB file limit (if you don't use the unlimited dimension)
        * often preffered for distributing products

    * netCDF 64 bit offset
        * supports larger files

    * NetCDF4
        * based on HDF5
        * compression
        * multiple unlimited variables
        * new types inc. user defined
        * herarchical groups
        
        

* Developed by Unidata-UCAR with the aim of storing climate model data (3D+time)
* Auxilary information about each variable can be added
* Readable text equivalent called CDL (use ncdump/ncgen)
* Can be used with Climate and Forecast (CF) data convention
http://cfconventions.org/

## Data model:

* Dimensions:describe the axes of the data arrays.
* Variables: N-dimensional arrays of data.
* Attributes: annotate variables or files with small notes or supplementary metadata.

Example for an ocean model dataset:

* Dimensions
    * lat
    * lon
    * depth
    * time
* Variable
    * Temperature
    * Salinity
* Global Attibutes
    * Geographic grid type
    * History



* Variable attributes (Temperature)
    * Long_name: "sea water temperature" 
    * Missing_value: 1.09009E36
    * Units: deg. C
    * range: -2:50

## Example dataset structure
http://xarray.pydata.org/en/stable/data-structures.html

<br>
<img src="../figures/dataset-diagram.png">


## Python packages
* The main Python interface to the netCDF C library is [netCDF4](http://unidata.github.io/netcdf4-python/) package
* In this tutorial, however, we will use a more high-level package that has a **pandas-like API**: [xarray](http://xarray.pydata.org/en/stable/index.html).
* Another good package is [iris](http://scitools.org.uk/iris/).

# Working with netCDF files using xarray

* Alternative to plain netCDF4 access from python. 

* Brings the power of pandas to environmental sciences, by providing N-dimensional variants of the core pandas data structures:

* worth using for multidimensional data even when not using 

| Pandas | xarray  |
|---|---|
| Series  | DataArray  |
| DataFrame  | Dataset  |


DataArray uses names of dimensions making it easier to track than by using axis numbers. It is possible to write:

`da.sel(time='2000-01-01')` or `da.mean(dim='time')`
intead of ``df.mean(0)``

HTML documentation: http://xarray.pydata.org/

Thus, xarray operations allow you to use names, not numbers!
Compare:
```python
# xarray style
>>> ds.sel(time='2018-01-12').max(dim='ensemble')

# numpy style
>>> array[[0, 1, 2, 3], :, :].max(axis=2)
```

(Taken from Stephan Hoyer's [ECMWF talk](https://docs.google.com/presentation/d/16CMY3g_OYr6fQplUZIDqVtG-SKZqsG8Ckwoj2oOqepU/edit#slide=id.g2b68f9254d_1_27))

### The main advantages of using xarray versus plain netCDF4 are:

* intelligent selection along labelled dimensions (and also indexes)
* groupby operations
* data alignment
* IO (netcdf)
* conversion from and to Pandas.DataFrames

In [ ]:
# Import everything that we are going to need
import pandas as pd
import xarray as xr
import numpy as np

## Import a local dataset

In [ ]:
GETM = xr.open_dataset('../data/cefas_GETM_nwes.nc4')
GETM

We can see that this file holds bathymetryic, height and tempearture data for a gridded area of the North West European Shelf

In [ ]:
GETM.dims

We can print these dimensions to see the extend of the netCDF

In [ ]:
GETM.latc
#GETM.lonc
#GETM.time
#GETM.level

In [ ]:
print(type(GETM.coords['latc']))
GETM.coords['latc'].shape

In [ ]:
# List name of dataset attributes
GETM.attrs.keys()

In [ ]:
# List variable names
GETM.data_vars.keys()

This tells  us that bathymetry is 2D, varying only wiht lat and lon. Height and temp are 4D, varying with time and model level as well

Extract variable from dataset

In [ ]:
temp = GETM['temp']
print(type( temp ))
temp.shape

Access variable attributes

In [ ]:
# print variable attributes

for at, val in temp.attrs.items():
    print(f'{at:<15}: {val}')

## Accessing data values

In [ ]:
temp[0, 0, 90, 100]

In [ ]:
temp.values[0, 0, 90, 100]

## Indexing and selecting data

From http://xarray.pydata.org/
<br>
<img src="../figures/xarray_indexing_table.png">

In [ ]:
#positional by integer
print( temp[0, 2, :, :].shape )

# positional by label
print( temp.loc['1996-02-02T01:00:00', :, :, :].shape )

# by name and integer
print( temp.isel(level=1, latc=90, lonc=100).shape )

# by name and label
print( temp.sel(time='1996-02-02T01:00:00').shape )
#temp.loc

### Define selection using nearest value

In [ ]:
#GETM.sel(level=1)['temp']
GETM['temp'].sel(level=2, lonc=-5.0, latc=50.0, method='nearest')

In [ ]:
# Use tolerance for label selection (lat=-50 should not yield data)
tol = 0.5

try:
    GETM['temp'].sel(level=1, lonc=-5.0, latc=-50.0, method='nearest', tolerance=tol)
except KeyError:
    print(f'ERROR: outside tolerance of {tol}')

## Plotting

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs

Define a general mapping function using cartopy

In [ ]:
#def make_map(ds, var='', title=None, units=None):
def make_map():
    # create figure and axes instances
    fig = plt.figure()
    ax = fig.add_subplot(111, projection=ccrs.Stereographic(central_latitude=60))
    #ax.coastlines(resolution='50m', linewidth=0.5)
    ax.set_extent([-10, 15, 49, 60], crs=ccrs.PlateCarree())
    
    gl = ax.gridlines(draw_labels=False)
    
    feature = cartopy.feature.NaturalEarthFeature(name='coastline',
                                                  category='physical',
                                                  scale='50m',
                                                  edgecolor='0.5',
                                                  facecolor='0.8')
    ax.add_feature(feature)
    return fig, ax

make_map();

In [ ]:
latc = GETM.coords['latc']
lonc = GETM.coords['lonc']

var = GETM.temp.sel(time='1996-02-02T01:00:00', level=21)

# create arrays of coordinates for contourf
# lon2d, lat2d = np.meshgrid(lonc, latc)

fig, ax = make_map()
# draw filled contours.
h = ax.contourf(lonc, latc, var, 50, cmap=plt.cm.coolwarm, transform=ccrs.PlateCarree())

# add colorbar.
cbar = fig.colorbar(h)
cbar.set_label(var.units)

# add title
ax.set_title(f'A slice of {var.long_name}');

In [ ]:
# But often, this will do
#GETM.temp.isel(time=0, level=0).plot();
GETM.temp.sel(time='1996-02-02T01:00:00', level=21).plot();

## Arithmetic operations

In [ ]:
top = GETM['temp'].isel(time=0, level=4)
bottom = GETM['temp'].isel(time=0, level=0)

diff = top - bottom

diff.plot()

### Calculate average along a dimension

In [ ]:
# average over time
time_ave = GETM['temp'].mean('time')

#average over time and level (vertical)
time_and_level_ave = GETM['temp'].mean(['time','level'])

time_and_level_ave.plot()

In [ ]:
#zonal average (vertical)
timelon_ave = GETM['temp'].mean(['time','lonc']).isel(level=4)

timelon_ave.plot()

Using **xarray** we have the data access power of **netCDF4** with all the intelligent selection, arithmetic, statistical methods and plotting of **pandas**

## A dataset can easily be saved to a netCDF file

In [ ]:
ds = GETM[['temp']].mean('time','level')
ds.to_netcdf('../data/temp_avg_level_time.nc')

## Exercise

* Extract the bathymetry
* Extract the time averaged seabed temperature (level=0)
* Produce a scatter plot of depth vs. seabed temperature


In [ ]:
GETM.data_vars.keys()

In [ ]:
# Your code here

In [ ]:
# Scroll down for a solution

In [ ]:
# First get the bathy and temp data
bathy=GETM['bathymetry']
temp_bott=GETM['temp'].mean('time').isel(level=0)

In [ ]:
bathy.plot();

In [ ]:
bedtemp.plot()

In [ ]:
# Make a simple scatter plot
fig, ax = plt.subplots()
ax.scatter(bathy,bedtemp,marker='.',s=1.0)
ax.set_xlabel('Bathymetry (m)')
ax.set_ylabel('Bottom temperature (deg. C)');

## Import remote dataset

xarray supports OpenDAP. This means that a dataset can be accessed remotely and subsetted as needed. Only the selected parts are downloaded.

In [ ]:
remote_data = xr.open_dataset(
      'http://iridl.ldeo.columbia.edu/SOURCES/.OSU/.PRISM/.monthly/dods',
      decode_times=False)
remote_data

# Excercise:
Import data from a netCDF or csv file and start exploring it. Some ideas:
- Use pandas to get quick statistics
- Do some data cleaning and calculations wtih numpy
- Plot it up with matplotlib, pandas, seaborn or cartopy as you prefer

I encourage you to use your own data if you have some. If not, we have some sample datasets you can explore:

### Earthquake data (csv file)
US Geological Survey (USGS) provides various [earthquakes data](https://earthquake.usgs.gov/data/data.php#eq) on a global scale. Its Earthquake Catalog contains earthquake source parameters (e.g. hypocenters, magnitudes, phase picks and amplitudes) and other products (e.g. moment tensor solutions, macroseismic information, tectonic summaries, maps) produced by contributing seismic networks.

If you follow this [link](http://earthquake.usgs.gov/earthquakes/search/), you can search throught the catalog and filter data by the magnitude, time and geographic region. In the `data/` folder, we provide an [example dataset](../data/earthquakes_2015_2016_gt45.csv) of earthquakes with magnitude >4.5 that occurred around the world over the period of a year.

To get you started, the following cell loads the data into a pandas DataFrame

In [ ]:
#import pandas as pd
#df = pd.read_csv('../data/earthquakes_2015_2016_gt45.csv', parse_dates = ['time',], index_col='time')
#df.head()

If you want to build your project on these data, some possible ideas are:
* `pandas` package will be most useful to read in the data, as well as analyse them
* Use `cartopy` to plot the data using longitude and latitude columns
* Explore `pandas`' `groupby()` method, which you can use to aggregate data by time or other parameter
* Create a histogram of earthquakes magnitude

### Arctic Sea Ice (netCDF files)
* In this project you are offered to use NOAA/NSIDC Climate Data Record of Passive Microwave Sea Ice Concentration.
* In the `../data/` directory, there are 2 netCDF files `seaice_conc_monthly*` that correspond to September 1991  and September 2012 .
* If you want to download data for other months, visit the [NSIDC's data portal](https://nsidc.org/data/search/#keywords=sea+ice/sortKeys=score,,desc/facetFilters=%257B%257D/pageNumber=1/itemsPerPage=25).

For this project, I recommend that you:
* use `xarray` for opening and reading the netCDF files
* use `cartopy` for creating a plot with a correct map projection
* use appropriate colormaps for the sea ice concentration and difference between the two years

Some code to get you started:

In [ ]:
#import cartopy.crs as ccrs
#import matplotlib.pyplot as plt
#import xarray as xr

In [ ]:
#ds1 = xr.open_dataset('../data/seaice_conc_monthly_nh_f08_199109_v02r00.nc')
#ds2 = xr.open_dataset('../data/seaice_conc_monthly_nh_f17_201209_v02r00.nc')

In [ ]:
## Code for creating a map
#fig = plt.figure()
#ax = fig.add_subplot(111, projection=ccrs.???(central_longitude=0))
#ax.coastlines(resolution='110m', linewidth=0.5)

#ax.gridlines()
#ax.set_extent([-180, 180, 40, 90], crs=ccrs.PlateCarree())